# Puzzle 3 — Presents go Pop!

## Sample Solutions

Sample solutions [Puzzle 3](https://leechristie.com/xmas24/3) of my [Christmas Coding Challenge](https://leechristie.com/xmas24).

## Loading the Data

For this puzzle we will pre-load the data into a list, for both parts we will need to refer to the list.

In [1]:
from decimal import Decimal

# for this puzzle sample solution I used Decimal type instead of float as best practice, since we'll be adding up money values
# although floating point errors shouldn't really affect us, alternatively we could just convert to pence and add integers
GIFT_VALUES : list[Decimal] = []

# standard code for looping over a file
with open('input3.txt') as file:
    for line in file:
        line = line.strip()

        # convert do Decimal type and append to the gift list
        line = Decimal(line)
        GIFT_VALUES.append(line)

# counting the gifts as we will use this later
# I will be using them as globals later, so I named the variables GIFT_VALUES and NUM_GIFTS in all caps (not required, just style)
NUM_GIFTS = len(GIFT_VALUES)

# global for the number of gifts we want to take
NUM_TO_TAKE = 8

# Part 1

For Part 1 you are being asked to implement a specific strategy rather than come up with your own idea.

The strategy Santa suggests is [a greedy algorithm](https://en.wikipedia.org/wiki/Greedy_algorithm), which is not a specific algorithm it's a general classification of problem-solving heuristics which typically get good, but not optimal solutions to difficult problems. It is often used where no efficient optimal algorithm is known.

So here you are testing on implementing the steps Santa outlined in the puzzle.

For this solution I chose not to pre-sort the list of items. Sorting would be $O(n\ \text{log}n)$, and making use of this would require adding a bunch of extra bookkeeping like tracking the position in a tuple.

I just loop over the list twice (implicitly with `max()` and `.index()`) each time we want to take a present, which happens a fixed number of time so this is $O(nk)$ where $k=8$ in our case the number of desired gifts to open. This has better complexity if the number of gifts $n$ grows and the number of gifts to choose $k$ stays the same. It could be improved by looping over once each time to get the arg max, but doing it twice required less code.

I can see the $O(n\ \text{log}n)$ solution would make sense too.

In [2]:
# clone the list as we will edit it to remove unavailable gifts
available_presents = GIFT_VALUES.copy()

# track the total value
taken_total = 0
token_count = 0

# loop until we have taken 8 gifts
while token_count < NUM_TO_TAKE:

    # get the maximum value, if 0 we ran out of gifts
    next_take = max(available_presents)
    assert next_take > 0

    # locate the index of the max gift
    next_index = available_presents.index(next_take)

    # track the chosen value
    taken_total += next_take
    token_count += 1

    # remove the left item from consideration (if we didn't take the leftmost gift)
    if next_index > 0:
        available_presents[next_index - 1] = 0

    # remove the current item from consideration
    available_presents[next_index] = 0

    # remove the right item from consideration (if we didn't take the rightmost gift)
    if next_index < NUM_GIFTS - 1:
        available_presents[next_index + 1] = 0
    
print(f"The total value of {NUM_TO_TAKE} gifts according to Santa's strategy is £{taken_total:.2f}")

The total value of 8 gifts according to Santa's strategy is £4738.61


# Part 2 - Optimum

This puzzle was designed to be more difficult to efficiently solve than the other puzzles. It's a variation of a Leetcode problem I saw someone describe on YouTube - I don't remember the video I saw, but here is a write-up from 2021 on [Medium by Prathamesh Kesarkar](https://medium.com/@pratham.kesarkar/leetcode-198-house-robber-cc3d13dcbaf7) which explains it.

My puzzle is slightly different as I modified the problem to add the restriction that you are choosing a specific number of elements the original had you take as many items as you want, which means you will need to keep track of one extra piece of information - remaining number of presents to take.

The solution below is a recursive solution where you call the function `best_total_value` giving the `lower_index` which is the first present you will **consider** taking (the base call will set this to 0, the first present, and the `gift_limit` is the number of presents you need to take it total.

On my computer, without using `@cache`, meaning we do a recursive call even if we have computed this calls before it takes about 5 seconds to run, making **54,696,377** recursive function calls.

Adding `@cache` it runs almost instantly as takes only **314** recursive calls. This is because using `@cache`, Python will automatically track which values of `lower_index` and `gift_limit` is has seen before and not call the function again for those values.

Alternatively, we can manually compute values for this is a table for $42 \times 8$ (number of values for `lower_index` and `gift_limit`), which would be a [Dynamic Programming](https://en.wikipedia.org/wiki/Dynamic_programming) solution. Using `@cache` in Python just automates this for us, with the slight downside that it is likely using hash tables in the implementation instead of an array of numbers that would be used in pure implementation of a dynamic programming solution.

I think it's easier to follow using recursion and `@cache`, but try implementing the pure Dynamic Programming solution to see.

An alternate, but slower method to solve the problem would be computing all possible [k-Combinations](https://en.wikipedia.org/wiki/Combination) for $n=42$ items and $k=8$ (there are **118,030,185**) then checking which don't have adjacent chosen elements, and checking which has the best max total.

If you try to brute force all possible length-42 Boolean strings to take or not take then it will be **4,398,046,511,104** combinations, which is likely infeasible.

In [3]:
from functools import cache

@cache
def best_total_value(gift_limit: int, lower_index: int=0) -> Decimal:

    # we don't need to declare globals if we are read-only, but I think this makes it clear they are used
    global NUM_GIFTS, GIFT_VALUES

    # check how many available gifts there are from the current considered index
    length = NUM_GIFTS - lower_index

    # Base Cases
    # ----------

    # if there are no gifts, or we hit the limit, best total is zero
    if length <= 0 or gift_limit <= 0:
        return Decimal('0.0')

    # if there is only one present left, we take it
    if length == 1:
        return GIFT_VALUES[lower_index]

    # Recursive Case
    # --------------

    # recursively find the best value we can get if we take this gift
    # this means we need to skip ahead 2 (because we can't take the next gift)
    # remember to add the value of this gift
    take_lower_value = GIFT_VALUES[lower_index] + best_total_value(gift_limit - 1, lower_index + 2)

    # recursively find the best value we can get if we don't take this gift
    # this means we only skip ahead 1 position because the next gift is still available
    # we don't add the value of this gift because we aren't taking it
    dont_take_lower_value = best_total_value(gift_limit, lower_index + 1)

    # return whichever is best
    return max((take_lower_value, dont_take_lower_value))

In this cell we simply make the top-level recursive call. This is when `lower_index=0` because we consider the entire list, and `gift_limit=8` because we haven't taken any gifts

In [4]:
optimal_val = best_total_value(gift_limit=NUM_TO_TAKE)

print(f"The total value of {NUM_TO_TAKE} gifts according to the optimal strategy is £{optimal_val:.2f}")

The total value of 8 gifts according to the optimal strategy is £4855.08
